# IMPORT DATA


In [1]:
import numpy as np
import pandas as pd
from utils import *
from sklearn.model_selection import train_test_split

db = pd.read_csv("data.csv")

## 1. Delete unused variables


In [2]:
repeat_columns = []
# unused and repeated metadata are dropped
repeat_columns += ["identifierHash", "type", "country", "gender", "civilityTitle"]
db1 = db.drop(repeat_columns, axis=1)
db1.head()

,language,socialNbFollowers,socialNbFollows,socialProductsLiked,productsListed,productsSold,productsPassRate,productsWished,productsBought,civilityGenderId,hasAnyApp,hasAndroidApp,hasIosApp,hasProfilePicture,daysSinceLastLogin,seniority,seniorityAsMonths,seniorityAsYears,countryCode
0,en,3,8,0,0,0,0.0,0,0,2,False,False,False,True,709,3205,106.83,8.9,us
1,de,3,8,0,0,0,0.0,0,0,2,False,False,False,True,709,3205,106.83,8.9,de
2,en,3,8,0,0,0,0.0,0,0,1,True,False,True,True,689,3205,106.83,8.9,se
3,en,3,8,0,0,0,0.0,0,0,2,False,False,False,True,709,3205,106.83,8.9,tr
4,en,3,8,0,0,0,0.0,0,0,1,True,False,True,True,709,3205,106.83,8.9,fr


## 2. Encode variables


In [3]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()

string_columns = [
    "language",
    "countryCode",
    "hasAnyApp",
    "hasAndroidApp",
    "hasIosApp",
    "hasProfilePicture",
]

for var in string_columns:
    var_cat = db[[var]]  # use double brakets to make sure i'm taking a dataframe
    var_cat_encoded = ordinal_encoder.fit_transform(var_cat)
    var_cat_df = pd.DataFrame(var_cat_encoded)
    var_cat_df.columns = [var + "_encoded"]
    db1 = db1.merge(var_cat_df, how="inner", left_index=True, right_index=True)

db2 = db1.drop(string_columns, axis=1)
db2.head()
db2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98913 entries, 0 to 98912
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   socialNbFollowers          98913 non-null  int64  
 1   socialNbFollows            98913 non-null  int64  
 2   socialProductsLiked        98913 non-null  int64  
 3   productsListed             98913 non-null  int64  
 4   productsSold               98913 non-null  int64  
 5   productsPassRate           98913 non-null  float64
 6   productsWished             98913 non-null  int64  
 7   productsBought             98913 non-null  int64  
 8   civilityGenderId           98913 non-null  int64  
 9   daysSinceLastLogin         98913 non-null  int64  
 10  seniority                  98913 non-null  int64  
 11  seniorityAsMonths          98913 non-null  float64
 12  seniorityAsYears           98913 non-null  float64
 13  language_encoded           98913 non-null  flo

In [4]:
# remove variables with no correlations
no_columns = ["seniority", "seniorityAsMonths", "seniorityAsYears"]
# week_columns=['language_encoded']
# unused_columns=no_columns+week_columns
db3 = db2.drop(no_columns, axis=1)

# HasBought As Predictor


Define X and Y


In [5]:
X = db3[
    [
        "socialNbFollowers",
        "socialNbFollows",
        "socialProductsLiked",
        "productsListed",
        "productsSold",
        "productsPassRate",
        "productsWished",
        "civilityGenderId",
        "daysSinceLastLogin",
        "language_encoded",
        "countryCode_encoded",
        "hasAnyApp_encoded",
        "hasAndroidApp_encoded",
        "hasIosApp_encoded",
        "hasProfilePicture_encoded",
    ]
]
# Y=db3['HasBought'] = db3['productsBought'].apply(lambda x: '1' if x >0 else '0')
Y = db3["HasBought"] = db3["productsBought"].apply(
    lambda x: "2" if x >= 3 else ("1" if x < 3 and x > 0 else "0")
)

Ont été appliqués deux modèles de classification (Naive Bayes, Machines à vecteurs de support), deux modèles de régression (Régression logistique, Arbre de décision) et deux modèles supervisés avancés (RandomForestClassifier, Xtreme gradient boosting) pour évaluer les performances des différents modèles.


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42
)
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)
models(X_train, Y_train, X_test, Y_test)

Select 1 : Naive Bayes, 2: Support Vector Machines, 3: Logistic Regression, 4: Decision Tree, 5: RandomForestClassifier,6: Extreme gradient boosting
testing set accuracy score:  0.9456834752772502
testing set accuracy mean:  0.9478052264018588
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     30803
           1       0.36      0.10      0.15      1388
           2       0.53      0.34      0.42       451

    accuracy                           0.95     32642
   macro avg       0.62      0.48      0.51     32642
weighted avg       0.92      0.95      0.93     32642

confusion matrix: 
[[30582   153    68]
 [ 1189   132    67]
 [  210    86   155]]
